<a href="https://colab.research.google.com/github/hyang0129/2022TSR/blob/main/TIMEREG_MHSA_AUG1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/hyang0129/TimeSeriesRegularization.git
!(cd TimeSeriesRegularization ; git checkout inceptiontime; git reset --hard HEAD; git pull)
# !(cd TimeSeriesRegularization ; git checkout main; git reset --hard HEAD; git pull)
!(cd TimeSeriesRegularization ; pip install -r requirements.txt -q)


fatal: destination path 'TimeSeriesRegularization' already exists and is not an empty directory.
Already on 'inceptiontime'
Your branch is up to date with 'origin/inceptiontime'.
HEAD is now at e68e509 added simple mhsa
Already up to date.
     |████████████████████████████████| 9.5 MB 4.3 MB/s 


In [2]:
import sys 
sys.path.append('TimeSeriesRegularization')

In [3]:
%load_ext autoreload


In [4]:
%autoreload

from tsr.datasets.datasetmanager import DatasetManager
from tsr.datasets import TEP_DatasetManager
from tsr.config import Config
import gdown
import pandas as pd
import numpy as np
from sklearn import preprocessing
from tqdm.autonotebook import tqdm
import tensorflow as tf
from loguru import logger
import pyreadr as py
from sklearn.model_selection import train_test_split, KFold
from tsr.utils import shell_exec
from tsr.tpusetup import connect_to_tpu
from tsr.datasets import UAE_DatasetManager
from tsr.models.attention import ConvMHSA


# df = pd.read_parquet('/content/drive/Shareddrives/KAGGLE/TSR/TEP/TEP_data.parquet', engine='pyarrow')



TimeSeriesRegularization/tsr/datasets/ngafid.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [5]:

config = Config.get_standard_config('TEP.yaml')
config.as_attr_dict()
dsm = UAE_DatasetManager(config)



2022-01-26 00:46:29.204 | DEBUG    | tsr.config:collect_sub_configs:141 - Sub Config Not Specified for hyperparameters config group
2022-01-26 00:46:29.212 | DEBUG    | tsr.config:collect_sub_configs:141 - Sub Config Not Specified for model config group
2022-01-26 00:46:29.213 | DEBUG    | tsr.config:__init__:62 - {'model': {'architecture_name': 'conv_mhsa', 'input_shape': [500, 52], 'conv_filters': [128, 128, 512], 'kernel': [7, 7, 7], 'strides': [1, 2, 2], 'num_heads': 8, 'd_model': 512, 'dff': 512}}
2022-01-26 00:46:29.217 | INFO     | tsr.config:collect_sub_configs:138 - Added Subconfig for model from TimeSeriesRegularization/tsr/configs/model/conv_mhsa_tep.yaml
2022-01-26 00:46:29.221 | DEBUG    | tsr.config:collect_sub_configs:141 - Sub Config Not Specified for augment config group
2022-01-26 00:46:29.226 | DEBUG    | tsr.config:collect_sub_configs:141 - Sub Config Not Specified for environment config group
2022-01-26 00:46:29.230 | DEBUG    | tsr.config:__init__:62 - {'hyperpara

In [6]:
strategy = connect_to_tpu()

TPU address is grpc://10.114.5.82:8470
Running on TPU  grpc://10.114.5.82:8470
INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.114.5.82:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.114.5.82:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [7]:

# def get_to_length(length):
#     def to_length(a): 
#         return np.pad(a, (0, np.max(length - len(a), 0)))
#     return to_length 

# def get_max_length(x):
#     return np.max(x.iloc[:, 0].apply(lambda x : len(x)))

# max = get_max_length(x)


# x = x.iloc[:, 0:].applymap(get_to_length(max))


In [8]:
for i in np.array(range(len(list(dsm.directories.keys())))):
    
    print(i, list(dsm.directories.keys())[i])

0 ArticularyWordRecognition
1 AtrialFibrillation
2 BasicMotions
3 Cricket
4 DuckDuckGeese
5 ERing
6 EigenWorms
7 Epilepsy
8 EthanolConcentration
9 FaceDetection
10 FingerMovements
11 HandMovementDirection
12 Handwriting
13 Heartbeat
14 LSST
15 Libras
16 MotorImagery
17 NATOPS
18 PEMS-SF
19 PenDigits
20 PhonemeSpectra
21 RacketSports
22 SelfRegulationSCP1
23 SelfRegulationSCP2
24 StandWalkJump
25 UWaveGestureLibrary


# GET DATASET

In [9]:
# results = []

# index = 1
# for i in np.array(range(len(list(dsm.directories.keys()))))[index:index+1]:

#     print(i, list(dsm.directories.keys())[i])

#     x, y = dsm.get_dataset_as_array(list(dsm.directories.keys())[i], split = 'TRAIN')
#     x_test, y_test = dsm.get_dataset_as_array(list(dsm.directories.keys())[i], split = 'TEST')

#     ds, test_ds, x = dsm.get_datasets_as_tf(list(dsm.directories.keys())[i])





# AUG

In [10]:
# from tsr.methods.augmentation import Cutmix, Mixup, Cutout 


# def get_augs(SHAPE, BATCH_SIZE = 64, DO_PROB=0.5, element_prob = 0.5):
    

#     mixup = Mixup(batch_size = BATCH_SIZE, 
#                 do_prob = DO_PROB,
#                 sequence_shape =  SHAPE[1:],
#                 linear_mix_min = 0.1,
#                 linear_mix_max = 0.5)
    
#     cutmix = Cutmix(batch_size = BATCH_SIZE, 
#                 do_prob = DO_PROB,
#                 sequence_shape =  SHAPE[1:],
#                 min_cutmix_len = SHAPE[1]//2,
#                 max_cutmix_len =  SHAPE[1],
#                 channel_replace_prob = element_prob,

#     )
#     cutmix.batch = BATCH_SIZE
#     cutout = Cutout(
#         batch_size = BATCH_SIZE, 
#         do_prob = DO_PROB,
#         sequence_shape =  SHAPE[1:],
#         min_cutout_len =  SHAPE[1]//2,
#         max_cutout_len =  SHAPE[1],
#         channel_drop_prob = element_prob,
#     )
#     cutout.batch = BATCH_SIZE


#     def batch_aug(x, y): 
#         example = {'input': x, 'target': y}
#         example = cutmix(example)
#         example = cutout(example)
#         example = mixup(example)
#         x, y = example['input'], example['target']
#         return x, y
#     return batch_aug

#     return batch_aug



# MODEL

In [11]:
from tsr.models.attention import EncoderLayer
from tsr.config import Config


def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                            np.arange(d_model)[np.newaxis, :],
                            d_model)

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

class PositionalEncoding(tf.keras.layers.Layer):

    def __init__(self, maximum_position_encoding = 1000, d_model = 256):
        super(PositionalEncoding, self).__init__()

        self.pos_encoding = positional_encoding(maximum_position_encoding,
                                        d_model)

    def call(self, x):

        seq_len = tf.shape(x)[1]
        x += self.pos_encoding[:, :seq_len, :]

        return x 



class ConvMHSA(tf.keras.Sequential):
    @classmethod
    def from_config(cls, config: Config):

        if config.hyperparameters.num_class > 2:
            output = tf.keras.layers.Dense(config.hyperparameters.num_class, activation="softmax")
        else:
            output = (tf.keras.layers.Dense(1, activation="sigmoid"),)

        mc = config.model
        return cls(
            input_shape=mc.input_shape,
            conv_filters=mc.conv_filters,
            kernel=mc.kernel,
            strides=mc.strides,
            num_heads=mc.num_heads,
            d_model=mc.d_model,
            dff=mc.dff,
            output=output,
        )

    def __init__(
        self,
        input_shape=(4096, 23),
        conv_filters=[128, 128, 256, 256, 512],
        kernel=[7, 7, 3, 7, 7],
        strides=[1, 2, 1, 2, 2],
        num_heads=8,
        d_model=512,
        dff=512,
        output: tf.keras.layers.Layer = tf.keras.layers.Dense(1, activation="sigmoid"),
    ):

        super().__init__(
            [
                tf.keras.Input(shape=input_shape),
            ]
            + [
                tf.keras.layers.Conv1D(
                    filters=conv_filters[i],
                    kernel_size=kernel[i],
                    strides=strides[i],
                    padding="same",
                    activation="relu",
                )
                for i in range(len(kernel))
            ] 
            +
            [
                tf.keras.layers.Conv1D(
                    filters=d_model,
                    kernel_size=13,
                    strides=1,
                    padding="same",
                    activation="tanh",
                )
            ]
            + [
                # tf.keras.layers.Activation('tanh'),
                PositionalEncoding(d_model = d_model),
                EncoderLayer(d_model=d_model, num_heads=num_heads, dff=dff),
                # EncoderLayer(d_model=d_model, num_heads=num_heads, dff=dff),
                # tf.keras.layers.GlobalAveragePooling1D(),
               tf.keras.layers.Lambda( lambda x : x[:, -1, :]), 
                output,
            ]
        )


In [12]:
import tensorflow as tf



class InceptionBlock(tf.keras.layers.Layer):


    def __init__(self,
                 nb_filters=32,
                 use_bottleneck=True,
                 kernel_size=41,
                 stride = 1,
                 depth = 3):

        super().__init__()
        self.nb_filters = nb_filters
        self.use_bottleneck = use_bottleneck
        self.kernel_size = kernel_size - 1
        self.callbacks = None
        self.bottleneck_size = 32
        self.stride = stride
        self.depth = depth


    def build(self, input_shape): 
        input_tensor = tf.keras.Input(input_shape[1:])

        x = input_tensor 
        shortcut_x = input_tensor

        for d in range(self.depth):
            stride = self.stride if d == self.depth - 1 else 1
            x = InceptionLayer(
                nb_filters =  self.nb_filters,
                use_bottleneck = self.use_bottleneck,
                kernel_size = self.kernel_size,
                stride = stride,
            )(x)

        x = ShortcutLayer(stride = self.stride)([x, shortcut_x])

        self.model = tf.keras.models.Model(inputs=input_tensor, outputs=x)


    def call(self, x):
        return self.model(x)


class InceptionLayer(tf.keras.layers.Layer):

    def __init__(self,
                 nb_filters=32,
                 use_bottleneck=True,
                 kernel_size=41,
                 stride = 1):

        super().__init__()
        self.nb_filters = nb_filters
        self.use_bottleneck = use_bottleneck
        self.kernel_size = kernel_size - 1
        self.callbacks = None
        self.bottleneck_size = 32
        self.stride = stride

    def build(self, input_tensor):
        activation = 'linear'
        stride = self.stride
        input_tensor = tf.keras.Input(input_tensor[1:])

        if self.use_bottleneck and int(input_tensor.shape[-1]) > 1:
            input_inception = tf.keras.layers.Conv1D(filters=self.bottleneck_size, kernel_size=1,
                                                  padding='same', activation=activation, use_bias=False)(input_tensor)
        else:
            input_inception = input_tensor

        # kernel_size_s = [3, 5, 8, 11, 17]
        kernel_size_s = [self.kernel_size // (2 ** i) for i in range(3)]

        conv_list = []

        for i in range(len(kernel_size_s)):
            conv_list.append(tf.keras.layers.Conv1D(filters=self.nb_filters, kernel_size=kernel_size_s[i],
                                                 strides=stride, padding='same', activation=activation, use_bias=False)(
                input_inception))

        max_pool_1 = tf.keras.layers.MaxPool1D(pool_size=3, strides=stride, padding='same')(input_tensor)

        conv_6 = tf.keras.layers.Conv1D(filters=self.nb_filters, kernel_size=1,
                                     padding='same', activation=activation, use_bias=False)(max_pool_1)

        conv_list.append(conv_6)

        x = tf.keras.layers.Concatenate(axis=2)(conv_list)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(activation='relu')(x)
        
        self.model = tf.keras.models.Model(inputs=input_tensor, outputs=x)

    def call(self, x):
        return self.model(x)
        

class ShortcutLayer(tf.keras.layers.Layer):

    def __init__(self, stride = 1):
        super().__init__()
        self.stride = stride

    def build(self, input_shapes):

        input_tensor, shortcut_tensor = input_shapes
        input_tensor, shortcut_tensor = (tf.keras.Input(input_tensor[1:]), tf.keras.Input(shortcut_tensor[1:]))

        shortcut_y = tf.keras.layers.Conv1D(filters = int(input_tensor.shape[-1]), kernel_size = 1,
                                            padding = 'same', use_bias = False)(shortcut_tensor)
        shortcut_y = tf.keras.layers.BatchNormalization()(shortcut_y)
        shortcut_y = tf.keras.layers.MaxPool1D(pool_size=self.stride, strides=self.stride, padding='same')(shortcut_y)
        
        x = tf.keras.layers.Add()([input_tensor, shortcut_y])
        x = tf.keras.layers.Activation('relu')(x)  

        self.model = tf.keras.Model(inputs=(input_tensor, shortcut_tensor), outputs=x)

    def call(self, inputs):
        return self.model(inputs)


def get_model(num_class=3):

    d_model = 512
    num_heads = 8
    dff = 512 

    model = tf.keras.Sequential([
        tf.keras.layers.Input((1751, 3)), 
        # tf.keras.layers.Conv1D(64, 3),
        # InceptionLayer(),
        InceptionBlock(stride=2),
        InceptionBlock(stride=2),
        tf.keras.layers.Conv1D(d_model, 1, activation = 'tanh'),
        PositionalEncoding(d_model = d_model),
        EncoderLayer(d_model=d_model, num_heads=num_heads, dff=dff),
        EncoderLayer(d_model=d_model, num_heads=num_heads, dff=dff),
        tf.keras.layers.Lambda( lambda x : x[:, -1, :]), 
        # tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(num_class, activation="softmax")
    ]
    )

    return model 

# model = get_model()    
# model.summary()

In [13]:


# with strategy.scope():
#     model = get_model(num_class=4)

#     model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5),
#             metrics = [
#                         tf.keras.metrics.SparseCategoricalAccuracy()
#             ], 
#             loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False))

# history = model.fit(
#     ds.map(get_augs(x , DO_PROB=0.9)), 
#     steps_per_epoch = 100, 
#     epochs = 100, 
#     validation_data = test_ds,
#     # callbacks = callbacks,
# )

# result = (i, list(dsm.directories.keys())[i], model.evaluate(test_ds)[1])
# results.append(result)

# TRAIN 

In [14]:
from tsr.methods.augmentation import Cutmix, Mixup, Cutout


def get_augs(SHAPE, BATCH_SIZE = 64, DO_PROB = 0.5, element_prob = 0.5, version = 0):
    if version == 0:
        DO_PROB = 0.7
        element_prob = 0.3

    elif version == 1:
        DO_PROB = 0.8
        element_prob = 0.5

    elif version == 2:
        DO_PROB = 0.8
        element_prob = 0.1

    mixup = Mixup(batch_size = BATCH_SIZE,
                  do_prob = DO_PROB,
                  sequence_shape = SHAPE[1:],
                  linear_mix_min = 0.1,
                  linear_mix_max = 0.5)

    cutmix = Cutmix(batch_size = BATCH_SIZE,
                    do_prob = DO_PROB,
                    sequence_shape = SHAPE[1:],
                    min_cutmix_len = SHAPE[1] // 2,
                    max_cutmix_len = SHAPE[1],
                    channel_replace_prob = element_prob,

                    )
    cutmix.batch = BATCH_SIZE
    cutout = Cutout(
        batch_size = BATCH_SIZE,
        do_prob = DO_PROB,
        sequence_shape = SHAPE[1:],
        min_cutout_len = SHAPE[1] // 2,
        max_cutout_len = SHAPE[1],
        channel_drop_prob = element_prob,
    )
    cutout.batch = BATCH_SIZE

    if version == 0:

        def batch_aug(x, y):
            example = {'input': x, 'target': y}
            example = cutmix(example)
            example = cutout(example)
            example = mixup(example)
            x, y = example['input'], example['target']
            return x, y

    elif version == -1:

        def batch_aug(x, y):
            return x, y

    elif version == 1:

        def batch_aug(x, y):
            example = {'input': x, 'target': y}
            example = cutmix(example)
            example = cutout(example)
            example = mixup(example)
            x, y = example['input'], example['target']
            return x, y

    elif version == 2:

        def batch_aug(x, y):
            example = {'input': x, 'target': y}
            example = cutmix(example)
            example = cutmix(example)
            example = cutout(example)
            # example = cutout(example)
            example = mixup(example)
            x, y = example['input'], example['target']
            return x, y

    else:
        raise KeyError('Augmentation Version Not Specified')

    return batch_aug



In [ ]:
from tsr.models.inceptiontime import Classifier_INCEPTION
from tsr.methods.augmentation import get_augs 
import os 
from tqdm.notebook import tqdm
from tsr.models.attention.simplemhsa import SimpleMHSA


DO_PROB = 0.7
element_prob = 0.3 
aug_version = 1
save_path = '/content/drive/Shareddrives/KAGGLE/TSR/AUGMENTATION PAPER 2022/RESULTS'
model_name = 'SimpleMHSA'
repeats = 10
start = 0


for seed in tqdm(range(start, repeats)):

    for i in tqdm(np.array(range(len(list(dsm.directories.keys()))))[0:]):

        dataset_name = list(dsm.directories.keys())[i]
        save_to_file = os.path.join(save_path, '_'.join([dataset_name, model_name, 'SEED%02.f' % seed, 'AUG%02.f' % aug_version, '.csv']))
        
        if not os.path.exists(save_to_file): 
            ds, test_ds, shape, num_classes = dsm.get_datasets_as_tf(list(dsm.directories.keys())[i], resample_seed = seed)

            if shape[1] < 513:

                with strategy.scope():

                    model = SimpleMHSA(input_shape=shape[1:], num_class=num_classes)

                    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4),
                                metrics = [
                                            tf.keras.metrics.SparseCategoricalAccuracy()
                                ], 
                                loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False))
                    
                    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=10,
                                                                min_lr=0.0001)
                    

                    early_stop = tf.keras.callbacks.EarlyStopping(
                        monitor='val_loss', min_delta=0, patience=100, verbose=0,
                        mode='auto', baseline=None, restore_best_weights=True
                    )

                    history = model.fit(
                        ds.map(get_augs(SHAPE = shape , BATCH_SIZE = 64, DO_PROB=DO_PROB, element_prob = element_prob, version = aug_version)), 
                        # ds,
                        # steps_per_epoch = max(int(shape[0]/64 +1), 20), 
                        steps_per_epoch = max(int(shape[0]/64 +1), 100), 
                        epochs = 1500, 
                        # epochs = 10,
                        validation_data = test_ds,
                        callbacks = [
                                    reduce_lr, early_stop
                        ],
                        verbose = False, 
                        # verbose=  True, 
                    )


                results = pd.DataFrame(history.history)

                dataset_name = list(dsm.directories.keys())[i]

                results['dataset_name'] = dataset_name
                results['model'] = model_name
                results['epoch'] = results.index
                results['do_prob'] = DO_PROB
                results['element_prob'] = element_prob
                results['aug_version'] = aug_version 
                results['seed'] = seed 
                results.to_csv(save_to_file) 


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sktime/utils/data_io.py:722: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data["dim_" + str(dim)] = instance_list[dim]


Instructions for updating:
Use fn_output_signature instead


Instructions for updating:
Use fn_output_signature instead


  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

In [ ]:
i = 0
for example in ds:
    i += 1 
    if i > 17:
        break

example



In [ ]:
def resample_train_test(x, y, x_test, y_test, seed=0):
        
    if not seed == 0: 
            
        combined_y = np.concatenate([y_test, y])
        combined_x = np.concatenate([x_test, x])

        np.random.seed(seed = seed)
        train_indices = []
        test_indices = []

        for cls in range(np.max(combined_y)+1):
            np.max(combined_y)

            arr = np.argwhere(combined_y == cls)


            np.random.shuffle(arr)

            train_index = arr[:len(np.argwhere(y == cls))]
            test_index = arr[len(np.argwhere(y == cls)):]
            train_indices.append(train_index)
            test_indices.append(test_index)

        train_indices = np.squeeze(np.concatenate(train_indices))
        test_indices = np.concatenate(test_indices)


        x = combined_x[train_indices]
        y = combined_y[train_indices]
        x_test = combined_x[test_indices]
        y_test = combined_x[test_indices]
        print(train_indices.shape)

    return x, y, x_test, y_test     

i = 0
print(i, list(dsm.directories.keys())[i])
x, y = dsm.get_dataset_as_array(list(dsm.directories.keys())[i], split = 'TRAIN')
x_test, y_test = dsm.get_dataset_as_array(list(dsm.directories.keys())[i], split = 'TEST')

x.shape, x_test.shape

x, y, x_test, y_test = resample_train_test(x, y, x_test, y_test, seed=1)
x.shape


In [ ]:

results = pd.DataFrame(history.history)

dataset_name = list(dsm.directories.keys())[i]

results['dataset_name'] = dataset_name
results['epoch'] = results.index
results


In [ ]:
np.max(history.history['val_sparse_categorical_accuracy'])